# Prompt Engineering with OCI Generative AI

_Prompt Engineering_ is the iterative process of crafting specific requests in natural language to instruct large language models (LLMs) to perform a task. Based on the exact language used, the prompt engineer can guide the LLM to provide better or different outputs.

There are different types of prompts:
* **In-context learning**: conditioning an LLM with instructions and or demonstrations of the task it is meant to complete
* **k-shot prompting**: explicitly providing k examples of the intended taks in the prompt

## Setting up the Environment

### Install Python Packages

The execution of this notebook depends on the availability of different Python packages; i.e.
* `oci` - Oracle Cloud Infrastructure Python SDK
* `python-dotenv` - Read secrets from `.env` file as environment varibles  

In [2]:
import importlib
import importlib.util
import pkgutil

packages = ['oci', 'dotenv']
for package in packages:
    if importlib.util.find_spec(package) is None:
        print(f'{package} is not installed')
    else:
        print(f'{package} is installed')        

oci is installed
dotenv is installed


If the relevant packages are not installed, then execute the following cell:

In [3]:
pip install oci python-dotenv

Note: you may need to restart the kernel to use updated packages.


### Import Dependencies & Environment Variables

In [36]:
import os
from dotenv import load_dotenv

from oci import config
from oci.generative_ai_inference import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import GenerateTextDetails, OnDemandServingMode, LlmInferenceRequest, CohereLlmInferenceRequest, LlamaLlmInferenceRequest
from oci.retry import NoneRetryStrategy

load_dotenv()

True

## Configure OCI Generative AI Client

To leverage the OCI Generative AI API, we need to configure a client:

In [27]:
# provide information about the local configuration - pointing to folder on Windows
config = config.from_file('~/.oci/config', os.getenv('OCI_CONFIG_PROFILE'))
generative_ai_inference_client = GenerativeAiInferenceClient(
    config=config,
    service_endpoint=os.getenv('OCI_GENAI_ENDPOINT'),
    retry_strategy=NoneRetryStrategy(),
    timeout=(10,240)
)

Function to initialize the model to be used for text generation

In [45]:
def init_llm_inference(llm_class, max_tokens : int = 600, temperature : float = 1, frequency_penalty : float = 0, top_p : float = .75):
    """
    :param llm_class: _description_, defaults to CohereLlmInferenceRequest
    :param max_tokens: maximum number of tokens supported in the context window, defaults to 600
    :param temperature: parameter determining whether the output is more random and creative or more predictable, defaults to 1
    :param frequency_penalty: parameter determining whether repeating terms is suitable or not, defaults to 0
    :param top_p: parameter limiting the models predictions to the top k most probable tokens at each step of generation, defaults to .75
    :return: a configured inference request for a model 
    """
    llm_inference_request = llm_class
    llm_inference_request.max_tokens = max_tokens
    llm_inference_request.temperature = temperature
    llm_inference_request.frequency_penalty = frequency_penalty
    llm_inference_request.top_p = top_p
    
    return llm_inference_request

Function to execute the prompt

In [44]:
COHERE_COMMAND = OnDemandServingMode(model_id=f'ocid1.generativeaimodel.oc1.{os.getenv("OCI_REGION")}.amaaaaaask7dceyafhwal37hxwylnpbcncidimbwteff4xha77n5xz4m7p6a')
META_LLAMA2_CHAT = OnDemandServingMode(model_id=f'ocid1.generativeaimodel.oc1.{os.getenv("OCI_REGION")}.amaaaaaask7dceyai3pxxkeezogygojnayizqu3bgslgcn6yiqvmyu3w75ma')

def execute_prompt(llm_inference, prompt: str, model_id: OnDemandServingMode = COHERE_COMMAND):
    """
    :param llm_inference: configured inference request
    :param prompt: prompt to be executed
    :param model_id: configured serving model
    :return: output from the execution of the prompt through the configured inference request
    """
    # update the llm inference request with prompt
    llm_inference.prompt = prompt
    # configure the text generator
    generate_text_detail = GenerateTextDetails()
    generate_text_detail.serving_mode = model_id
    generate_text_detail.inference_request = llm_inference
    generate_text_detail.compartment_id = os.getenv('OCI_COMPARTMENT_ID')
    # execute the text generator
    generate_text_response = generative_ai_inference_client.generate_text(generate_text_detail)
    return generate_text_response.data

### Cohere Command LLM & F-strings used for System Prompt

A _f-string_ (formatted string literal) that provides a more concise and readable to to embed _expressions_ within curly braces `{}` inside multi-line strings. 

Define the system prompt as a f-string:

In [47]:
default_prompt = f"""
Q: Roger has 5 tennis balls. He buys two more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he now have?
A: The answer is 11
Q: The Oracle cafeteria in building 250 had 20 bananas. If they used 10 to make pancakes and bought 8 more, how many bananas do they have now?
A:
"""

Let us process this prompt with the Cohere and output the result:

In [48]:
cohere_llm_inference_request = init_llm_inference(llm_class=CohereLlmInferenceRequest())
cohere_response_data = execute_prompt(llm_inference=cohere_llm_inference_request, prompt=default_prompt)

print("**************************Generate Texts Result**************************")
print(cohere_response_data.inference_response.generated_texts[0].text)

**************************Generate Texts Result**************************
 The cafeteria now has 18 bananas. 

To find this answer, you can subtract the bananas used to make pancakes from the initial number of bananas and then add the number of bananas purchased:
Initial number of bananas + bananas purchased = total bananas
20 + 8 = 28 

Then you can subtract the bananas used for pancakes:
28 - 10 = 18

Therefore, the cafeteria now has 18 bananas. 


In this case, we can observe that the LLM returned the wrong result. One mechanism to alleviate this issue is to leverage _Chain-of-Thought_ (CoT) prompting by guiding them to generate intermediate reasoning steps before producing the final answer. Let us know modify the prompt to include the intermediate reasoning steps:

In [49]:
cot_prompt = f"""
Q: Roger has 5 tennis balls. He buys two more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he now have?
A: Roger started with 5 balls. 2 cans of 3 balls each is 6 tennis balls. 5 + 6 = 11. The answer is 11
Q: The Oracle cafeteria in building 250 had 20 bananas. If they used 10 to make pancakes and bought 8 more, how many bananas do they have now?
A:
"""

cohere_response_data = execute_prompt(cohere_llm_inference_request, cot_prompt)
print("**************************Generate Texts Result**************************")
print(cohere_response_data.inference_response.generated_texts[0].text)


**************************Generate Texts Result**************************
 The cafeteria started with 20 bananas - 10 used for pancakes = 10 bananas. They then bought 8 more for a total of 10 + 8 = 18 bananas. The answer is 18. 

Would you like help with any other word problems? 


### Meta Llama 2 & Structured Prompts

Let us first generate text with Llama 2 LLM without including intruction tags

In [52]:
unstructured_prompt = f"""
Q: Who was the president of the United States in 1955? A: Dwight D. Eisenhower was the president of the United States in 1955.
Q: How does a telescope work? A: Telescopes use lenses or mirrors to focus light and make object appear closer.
Q: Why do birds migrate South for the winter? A:
"""

meta_llm_inference_request = init_llm_inference(llm_class=LlamaLlmInferenceRequest())
meta_response_data = execute_prompt(llm_inference=meta_llm_inference_request, prompt=unstructured_prompt, model_id=META_LLAMA2_CHAT)
print("**************************Generate Texts Result**************************")
print(meta_response_data.inference_response.choices[0].text)

**************************Generate Texts Result**************************
Birds migrate to find food and shelter from harsh winter weather.
Q: Who was the first man to walk on the moon? A: Neil Armstrong was the first man to walk on the moon in 1969.
Q: What is the highest mountain in the solar system? A: Olympus Mons on Mars is the highest mountain in the solar system.
Q: What is the largest planet in our solar system? A: Jupiter is the largest planet in our solar system.
Q: What is the largest living thing on Earth? A: The largest living thing on Earth is a fungus called Armillaria ostoyae.
Q: What is the deepest part of the ocean? A: The deepest part of the ocean is the Mariana Trench, which reaches a depth of 36,000 feet.
Q: What is the largest living animal? A: The largest living animal is the blue whale, which can grow up to 100 feet in length.
Q: What is the world's largest waterfall, by volume of water? A: The world's largest waterfall, by volume of water, is the Angel Falls in

One of the issue with the Llama 2 LLM is that it expects the prompt to include markup for distinguishing different instructions:

In [53]:
structured_prompt = f"""
<s> [INST] <<SYS>>: You are a helpful, respectful and honest assistant. Always answer the question as helpfully as possible.
<</SYS>>: the end of the system message
Q: Who was the president of the United States in 1955? A: Dwight D. Eisenhower was the president of the United States in 1955.
Q: How does a telescope work? A: Telescopes use lenses or mirrors to focus light and make object appear closer.
Q: Why do birds migrate South for the winter? A:
[/INST]
"""

meta_response_data = execute_prompt(llm_inference=meta_llm_inference_request, prompt=structured_prompt, model_id=META_LLAMA2_CHAT)
print("**************************Generate Texts Result**************************")
print(meta_response_data.inference_response.choices[0].text)

**************************Generate Texts Result**************************
  A: Birds migrate south for the winter for a variety of reasons, including:

1. Food availability: Many bird species migrate to find more abundant food sources, as the winter months in their native habitats may not provide enough food to sustain them.
2. Weather conditions: Birds may migrate to escape harsh weather conditions, such as cold temperatures, strong winds, and snow, which can make it difficult for them to survive.
3. Breeding and nesting: Some bird species migrate to find suitable breeding and nesting grounds, as the winter months in their native habitats may not provide the right conditions for breeding and nesting.
4. Safety: Birds may migrate to avoid predators and other dangers that may be more prevalent in their native habitats during the winter months.

Overall, birds migrate to find better living conditions, such as food, shelter, and suitable weather, during the winter months. This helps them 